<a href="https://colab.research.google.com/github/chrober24/A.UD_289.84_seminar/blob/main/stylegan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
#@title Connect to Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Setup

import os
if os.path.isdir("/content/drive/MyDrive/SG3"):
    %cd "/content/drive/MyDrive/SG3/stylegan3"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir SG3
    %cd SG3
    !git clone https://github.com/dvschultz/stylegan3
    %cd stylegan3
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !mkdir raw_imgs
    !mkdir results
    !mkdir generated
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan3
    %cd stylegan3/
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !mkdir raw_imgs
    !mkdir results
    !mkdir generated
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja opensimplex torch==1.7.1 torchvision==0.8.2

#Training

In [ ]:
#@title Download pretrained model from Nvidia

model_name = 'stylegan3-t-ffhqu-1024x1024.pkl' #@param ['stylegan3-t-ffhq-1024x1024.pkl', 'stylegan3-t-ffhqu-1024x1024.pkl', 'stylegan3-t-ffhqu-256x256.pkl', 'stylegan3-r-ffhq-1024x1024.pkl', 'stylegan3-r-ffhqu-1024x1024.pkl', 'stylegan3-r-ffhqu-256x256.pkl', 'stylegan3-t-metfaces-1024x1024.pkl', 'stylegan3-t-metfacesu-1024x1024.pkl', 'stylegan3-r-metfaces-1024x1024.pkl', 'stylegan3-r-metfacesu-1024x1024.pkl', 'stylegan3-t-afhqv2-512x512.pkl', 'stylegan3-r-afhqv2-512x512.pkl']
url = 'https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/' + model_name
outpath = './pretrained/' + model_name

!wget $url -O $outpath

In [ ]:
#@title Prepare Raw Images

#options
source_folder = '/content/drive/MyDrive/stylegan2-aug-colab/stylegan2/raw_images/forest_gothic' #@param {type: 'string'}
dataset_name = 'forest_gothic_512x512' #@param {type: 'string'}
dest = './datasets/' + dataset_name +'.zip'
crop_type = 'center-crop' #@param ['center-crop', 'center-crop-wide']
resolution = '512x512' #@param ['256x256', '512x512', '1024x1024']

#prepare zip file of images cropped and formatted for stylegan training
!python dataset_tool.py --source=$source_folder  --dest=$dest --transform=$crop_type --resolution=$resolution

In [ ]:
#@title Run Training 
#@markdown When resuming from pretrained model, make sure the cfg matches that of the pretrained model
cfg = 'stylegan3-t' #@param ['stylegan3-t', 'stylegan3-r', 'stylegan2']
dataset = 'insects_machines' #@param {type: 'string'}
dataset_path = './datasets/' + dataset +'.zip'
resume_pkl = 'latest' #@param {type: 'string'}


!python train.py --outdir=./results --cfg=$cfg \
  --data=$dataset_path --gpus=1 --batch=4 --batch-gpu=4 --gamma=10 --snap=1 --kimg=5000 --metrics=none\
  --resume=$resume_pkl

# Generate Images

In [ ]:
#@title Choose a model
network_path = '/content/drive/MyDrive/SG3/stylegan3/results/00002-stylegan3-r-forest_gothic_512x512-gpus1-batch8-gamma10/network-snapshot-000100.pkl' #@param {type: 'string'}

In [ ]:
!python gen_images.py --outdir=./generated/1 --trunc=0.8 --seeds=1-100 \
    --network=$network_path

In [ ]:
#@title Generate images from seeds

out_path = './generated/1' #@param {type: 'string'}
truncation_val = 0 #@param {type: 'slider', min:0, max:5, step:0.1}
seeds = '1-100' #@param {type: 'string'}

!python gen_images.py --outdir=$out_path --trunc=$truncation_val \
--seeds=$seeds --network=$network_path \

In [ ]:
#@title Generate Linear Interpolation Video
video_name = 'lerp' #@param {type: 'string'}
output_path = './generated/' + video_name + '.mp4'

truncation_val = 0 #@param {type: 'slider', min:0, max:5, step:0.1}
seeds = '1-100' #@param {type: 'string'}
gridX = 1 #@param {type: 'slider', min:1, max:8, step:1}
gridY = 1 #@param {type: 'slider', min:1, max:8, step:1}
grid = gridX + 'x' + gridY

!python gen_video.py --output=$output_path --trunc=$truncation_val --seeds=$seeds --grid=$grid \
    --network=$network_path